# Azure Cache for Redis Vector Database

This sample shows how to connect with an existing Redis Database with RediSearch installed, create embeddings with openAI,
create indexes, load the vectors to the Vector Database, and query the top k results

## Prequisites

Set up your Redis database that we will use for a Vector Database using Azure Cache [Azure Cache with Redis](https://learn.microsoft.com/en-us/azure/azure-cache-for-redis/quickstart-create-redis-enterprise)

1. On the Advanced page, select the Modules drop-down and select RediSearch. This will create your cache with the RediSearch module installed. You can also install the other modules if you’d like. 
2. In the Zone redundancy section, select Zone redundant (recommended). This will give your cache even greater availability.  
3. In the Non-TLS access only section select Enable. 
4. For Clustering Policy, select Enterprise. RediSearch is only supported using the Enterprise cluster policy.  
5. Select Review + create and finish creating your cache instance. 

- Install Python libraries using `pip install -r requirements.txt`
- Enter your Redis environment variables for REDIS_HOST, REDIS_PORT, REDIS_PASSWORD in example.env
- Enter your openAI environment variables in example.env

In [1]:
import json
import requests
import numpy as np
import pandas as pd
import openai
from dotenv import dotenv_values
from redis import Redis
from redis.commands.search.field import VectorField, TextField, TagField
from redis.commands.search.query import Query
from redis.commands.search.result import Result

### Load environment variables and keys

In [36]:
# specify the name of the .env file name 
env_name = "example.env"
config = dotenv_values(env_name)

redis_host = config['REDIS_HOST']
redis_port = config['REDIS_PORT']
redis_passwd = config['REDIS_PASSWORD']

openai_api_key = config['openai_api_key']
openai_api_base = config['openai_api_base']
openai_api_version = config['openai_api_version']
openai_deployment_embedding = config['openai_deployment_embedding']

ITEM_KEYWORD_EMBEDDING_FIELD='item_keyword_vector'

# We are using text-embedding-ada-002
embedding_length = 1536

### Establish a connection to the database

In [15]:
redis_conn = Redis(
  host=redis_host,
  port=redis_port,
  password=redis_passwd,
)
print('Connected to redis')

Connected to redis


### Prep the data

In [16]:
df = pd.read_csv('../../Dataset/Reviews_small.csv')

In [17]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [18]:
NUMBER_PRODUCTS = len(df['Id'])

### Create content and generate embeddings using OpenAI text-embedding-ada-002

In [19]:
# We will combine productid, score, and text into a single field to run embeddings on
df['combined'] = 'productid: ' + df['ProductId'] + ' ' + 'score: ' + df['Score'].astype(str) + ' ' + 'text: ' + df['Text']
df['combined'].head()

0    productid: B001E4KFG0 score: 5 text: I have bo...
1    productid: B00813GRG4 score: 1 text: Product a...
2    productid: B000LQOCH0 score: 4 text: This is a...
3    productid: B000UA0QIQ score: 2 text: If you ar...
4    productid: B006K2ZZ7K score: 5 text: Great taf...
Name: combined, dtype: object

In [20]:
openai.api_type = "azure"
openai.api_key = openai_api_key
openai.api_base = openai_api_base
openai.api_version = openai_api_version

def createEmbeddings(text):
    response = openai.Embedding.create(input=text , engine=openai_deployment_embedding)
    embeddings = response['data'][0]['embedding']
    return embeddings

df['embedding'] = None
# iterate over the dataframe and create embeddings for each row
for index, row in df.iterrows():
    df.at[index, 'embedding'] = createEmbeddings(row['combined'])
    
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,combined,embedding
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,productid: B001E4KFG0 score: 5 text: I have bo...,"[-0.005925467703491449, -0.0029136091470718384..."
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,productid: B00813GRG4 score: 1 text: Product a...,"[-0.025345874950289726, -0.015591269358992577,..."
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,productid: B000LQOCH0 score: 4 text: This is a...,"[0.011405655182898045, 0.007502448279410601, -..."
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,productid: B000UA0QIQ score: 2 text: If you ar...,"[0.008057798258960247, 0.006768684834241867, -..."
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,productid: B006K2ZZ7K score: 5 text: Great taf...,"[-0.023589162155985832, -0.009200308471918106,..."


### Create an index on Id and insert our dataframe to the collection

In [21]:
def create_flat_index (redis_conn, vector_field_name, number_of_vectors, vector_dimensions=512, distance_metric='L2'):
    redis_conn.ft().create_index([
        VectorField(vector_field_name, "FLAT", {"TYPE": "FLOAT32", "DIM": vector_dimensions, "DISTANCE_METRIC": distance_metric, "INITIAL_CAP": number_of_vectors, "BLOCK_SIZE": number_of_vectors }),
        TagField("Id"),
        TagField("ProductId"),
        TagField("UserId"),
        TagField("ProfileName"),
        TagField("HelpfulnessNumerator"),
        TagField("HelpfulnessDenominator"),
        TagField("Score"),
        TagField("Time"),
        TagField("Summary"),
        TextField("Text"),
        TextField("combined"),
    ])

In [22]:
# Flush all data
redis_conn.flushall()

# Create flat index
create_flat_index(redis_conn, ITEM_KEYWORD_EMBEDDING_FIELD, NUMBER_PRODUCTS, embedding_length,'COSINE')

### Store the embeddings in Redis Vector Database

[Azure Cache with Redis](https://learn.microsoft.com/en-us/azure/azure-cache-for-redis/quickstart-create-redis-enterprise) provides a simple interface to create a vector database, store and retrieve data using vector search. You can read more about Vector search [here](https://mlops.community/vector-similarity-search-from-basics-to-production/). Additionally, here is a [blog post](https://lablab.ai/t/efficient-vector-similarity-search-with-redis-a-step-by-step-tutorial) demonstrating flat index vs. HNSW. This sample uses flat index.

In [23]:
def load_vectors(client:Redis, product_metadata, vector_dict, vector_field_name):
    p = client.pipeline(transaction=False)
    for index in product_metadata.keys():    
        # Hash key
        key = 'product:'+ str(index)+ ':' + product_metadata[index]['UserId']
        
        # Hash values
        item_metadata = product_metadata[index]
        item_keywords_vector = np.array(vector_dict[index]).astype(np.float32).tobytes()
        item_metadata[vector_field_name] = item_keywords_vector
        
        # HSET
        p.hset(key, mapping=item_metadata)
            
    p.execute()

In [24]:
product_metadata = df.drop('embedding', axis=1).head(NUMBER_PRODUCTS).to_dict(orient='index')

In [25]:
product_metadata[0]

{'Id': 1,
 'ProductId': 'B001E4KFG0',
 'UserId': 'A3SGXH7AUHU8GW',
 'ProfileName': 'delmartian',
 'HelpfulnessNumerator': 1,
 'HelpfulnessDenominator': 1,
 'Score': 5,
 'Time': 1303862400,
 'Summary': 'Good Quality Dog Food',
 'Text': 'I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.',
 'combined': 'productid: B001E4KFG0 score: 5 text: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.'}

In [26]:
load_vectors(redis_conn, product_metadata, df['embedding'], ITEM_KEYWORD_EMBEDDING_FIELD)

### User Query

In [27]:
userQuestion = "Great taffy"
retrieve_k = 3 # Retrieve the top 3 documents from vector database

In [28]:
# Generate embeddings for the question and retrieve the top k document chunks
questionEmbedding = createEmbeddings(userQuestion)
questionEmbedding = np.array(questionEmbedding).astype(np.float32).tobytes()

In [29]:
# Prepare the query
q = Query(f'*=>[KNN {retrieve_k} @{ITEM_KEYWORD_EMBEDDING_FIELD} $vec_param AS vector_score]').sort_by('vector_score').paging(0, retrieve_k).return_fields(
        'Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time',
        'Summary', 'Text', 'combined', 'vector_score',
).dialect(2)
params_dict = {"vec_param": questionEmbedding}

# Execute the query
results = redis_conn.ft().search(q, query_params=params_dict)

### Retrieve text from database

In [30]:
df_retrieved = pd.DataFrame()
for product in results.docs:
    print('***************Product  found ************')
    print(product.combined)
    print('vector_score: ', product.vector_score)
    
    df_retrieved = pd.concat([df_retrieved, pd.DataFrame([product.__dict__], columns=product.__dict__.keys())])

***************Product  found ************
productid: B006K2ZZ7K score: 5 text: Great taffy at a great price.  There was a wide assortment of yummy taffy.  Delivery was very quick.  If your a taffy lover, this is a deal.
vector_score:  0.110626459122
***************Product  found ************
productid: B006K2ZZ7K score: 5 text: This taffy is so good.  It is very soft and chewy.  The flavors are amazing.  I would definitely recommend you buying it.  Very satisfying!!
vector_score:  0.112391650677
***************Product  found ************
productid: B006K2ZZ7K score: 5 text: This saltwater taffy had great flavors and was very soft and chewy.  Each candy was individually wrapped well.  None of the candies were stuck together, which did happen in the expensive version, Fralinger's.  Would highly recommend this candy!  I served it at a beach-themed party and everyone loved it!
vector_score:  0.115778684616


In [31]:
df_retrieved

,id,payload,vector_score,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,combined
0,product:4:A1UQRSCLF8GW1T,None,0.110626459122,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,productid: B006K2ZZ7K score: 5 text: Great taf...
0,product:7:A3JRGQVEQN31IQ,None,0.112391650677,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...,productid: B006K2ZZ7K score: 5 text: This taff...
0,product:6:A1SP2KVKFXXRU1,None,0.115778684616,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...,productid: B006K2ZZ7K score: 5 text: This salt...


## OPTIONAL: Offer Response to User's Question
To offer a response, one can either follow a simple prompting method as shown below or leverage ways used by other libraries, such as [langchain](https://python.langchain.com/en/latest/index.html).

In [32]:
# create a prompt template 
template = """
    context :{context}
    Answer the question based on the context above. Provide the product id associated with the answer as well. If the
    information to answer the question is not present in the given context then reply "I don't know".
    Query: {query}
    Answer: """

In [33]:
# Create context for the prompt by combining the productid, score, and text of retrieved rows
df_retrieved['combined'] = 'productid: ' + df_retrieved['ProductId'] + ' ' + 'score: ' + df_retrieved['Score'].astype(str) + ' ' + 'text: ' + df_retrieved['Text']
context = '\n'.join(df_retrieved['combined'])

print(context)

productid: B006K2ZZ7K score: 5 text: Great taffy at a great price.  There was a wide assortment of yummy taffy.  Delivery was very quick.  If your a taffy lover, this is a deal.
productid: B006K2ZZ7K score: 5 text: This taffy is so good.  It is very soft and chewy.  The flavors are amazing.  I would definitely recommend you buying it.  Very satisfying!!
productid: B006K2ZZ7K score: 5 text: This saltwater taffy had great flavors and was very soft and chewy.  Each candy was individually wrapped well.  None of the candies were stuck together, which did happen in the expensive version, Fralinger's.  Would highly recommend this candy!  I served it at a beach-themed party and everyone loved it!


In [34]:
prompt = template.format(context=context, query=userQuestion)
print(prompt)


    context :productid: B006K2ZZ7K score: 5 text: Great taffy at a great price.  There was a wide assortment of yummy taffy.  Delivery was very quick.  If your a taffy lover, this is a deal.
productid: B006K2ZZ7K score: 5 text: This taffy is so good.  It is very soft and chewy.  The flavors are amazing.  I would definitely recommend you buying it.  Very satisfying!!
productid: B006K2ZZ7K score: 5 text: This saltwater taffy had great flavors and was very soft and chewy.  Each candy was individually wrapped well.  None of the candies were stuck together, which did happen in the expensive version, Fralinger's.  Would highly recommend this candy!  I served it at a beach-themed party and everyone loved it!
    Answer the question based on the context above. Provide the product id associated with the answer as well. If the
    information to answer the question is not present in the given context then reply "I don't know".
    Query: Great taffy
    Answer: 


In [35]:
response = openai.Completion.create(
    engine= config["openai_deployment_completion"],
    prompt=prompt,
    max_tokens=1024,
    n=1,
    stop=None,
    temperature=1,
)

print(response['choices'][0]['text'])

 Productid: B006K2ZZ7K, This taffy is so good. It is very soft and chewy. The flavors are amazing. I would definitely recommend you buying it. Very satisfying!!
